In [2]:
import os
import cloudinary
import cloudinary.uploader
import firebase_admin
from firebase_admin import credentials, firestore

In [3]:
# 🔐 Cloudinary config
cloudinary.config(
  cloud_name='dglfvvzg1',
  api_key='555873354438444',
  api_secret='_ELGOWW6c3l9-e9-94H_GrhoSfk'
)

In [4]:
# 🔐 Firebase config
cred = credentials.Certificate("../../private_key.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [5]:
# 📁 Imagens locais
IMAGES_FOLDER = r'C:\Users\Layanny\Documents\patrimonygo\src\assets\Patrimonios'

In [6]:
def agrupar_por_nome_base(pasta):
    grupos = {}
    for nome in os.listdir(pasta):
        if nome.lower().endswith(('.jpg', '.jpeg', '.png', '.webp')):
            base = ' '.join(nome.rsplit(' ', 1)[0].split('.'))  # Ex: BASILICA EMBARE
            grupos.setdefault(base, []).append(nome)
    return grupos


In [8]:
def enviar_e_atualizar():
    grupos = agrupar_por_nome_base(IMAGES_FOLDER)

    for patrimonio, imagens in grupos.items():
        print(f'\n📍 {patrimonio}')
        urls = []

        for img in imagens:
            path = os.path.join(IMAGES_FOLDER, img)
            try:
                response = cloudinary.uploader.upload(path, folder='patrimonios')
                url = response['secure_url']
                urls.append(url)
                print(f'  ✅ {img} -> {url}')
            except Exception as e:
                print(f'  ❌ Erro ao enviar {img}: {e}')
        
        # Atualiza Firestore com as URLs
        snapshot = db.collection('patrimonios_santos').where('nm_patrimonio', '==', patrimonio).get()
        if snapshot:
            doc_id = snapshot[0].id
            db.collection('patrimonios_santos').document(doc_id).update({'imageUrls': urls})
            print(f'  🔁 Firestore atualizado: {doc_id}')
        else:
            print(f'  ⚠️ Documento não encontrado para: {patrimonio}')

if __name__ == '__main__':
    enviar_e_atualizar()


📍 ALFÂNDEGA DA RECEITA FEDERAL DO BRASIL EM SANTOS
  ✅ ALFÂNDEGA DA RECEITA FEDERAL DO BRASIL EM SANTOS 01.jpg -> https://res.cloudinary.com/dglfvvzg1/image/upload/v1749607587/patrimonios/krs2efszqjgm9mugvqc6.jpg
  ✅ ALFÂNDEGA DA RECEITA FEDERAL DO BRASIL EM SANTOS 01.png -> https://res.cloudinary.com/dglfvvzg1/image/upload/v1749607588/patrimonios/soydha49c9bviftx6emu.png
  ✅ ALFÂNDEGA DA RECEITA FEDERAL DO BRASIL EM SANTOS 03.jpg -> https://res.cloudinary.com/dglfvvzg1/image/upload/v1749607589/patrimonios/h5nhhhyzrfqtgjcdisk6.jpg
  🔁 Firestore atualizado: kKEFGjaKqs57NVDbSl6I

📍 BASILICA EMBARE
  ✅ BASILICA EMBARE 01.jpg -> https://res.cloudinary.com/dglfvvzg1/image/upload/v1749607590/patrimonios/mw5auxigkwek0iwbgsdk.jpg
  ✅ BASILICA EMBARE 02.jpg -> https://res.cloudinary.com/dglfvvzg1/image/upload/v1749607591/patrimonios/wjdmt9czxmirfov2ljog.jpg
  ✅ BASILICA EMBARE 03.jpg -> https://res.cloudinary.com/dglfvvzg1/image/upload/v1749607592/patrimonios/gwqufa6imqbrzy87nfcm.jpg
  ⚠️ Docu